In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from distances import hello

hello()

'alv'

In [4]:
import pandas as pd
import numpy as np

In [5]:
# Datasets
msa_regions_pdbs = pd.read_csv('../datasets/msa_regions_pdbs.tsv', sep= '\t')
mapping_coordinates_uniprot_pdb = pd.read_csv('../datasets/mapping_coordinates_uniprot_pdb.tsv', sep = '\t')

In [6]:
# Tomar todas las distancias de los atomos pesados, no filtrar por threshold
def distances_all_atoms(path, output, pdb_coord): # threshold = 6
  '''
  Given a pdb file, calculates the interatomic distances among heavy atoms
  Returns: a data frame with interatomic distances
  path: path containing PDB files
  output: output path
  pdb_coord: pandas DataFrame containing regions in PDB coordinates
  '''
  import os
  import distances
  from distances import all_atoms_selector
  from Bio.PDB.PDBParser import PDBParser
  import warnings

  warnings.simplefilter("ignore")

  for pdb_id in pdb_coord.pdb.unique():
    f = os.path.join(path, f"{pdb_id}.pdb") # is the complete path to the pdb file
    if os.path.isfile(f):
      try:
        if pdb_id in pdb_coord.pdb.unique().tolist():
          output_file = output + pdb_id + ".csv"
          # iterate over each chain in the dataframe
          chains_kd_cre = {}
          chains = pdb_coord[pdb_coord.pdb == pdb_id].chain.drop_duplicates()
          for chain in chains:
            df_chains = pdb_coord[(pdb_coord.pdb == pdb_id) & (pdb_coord.chain == chain)]
            pdb_positions = set(eval(df_chains.mapping.values[0]).values()) # tomar el primero, ya que ese pdb puede estar en distintos MSAs
            kd = set()
            cre = set()
            kd_start = df_chains.pdb_start_kd.values[0]
            kd_end = df_chains.pdb_end_kd.values[0]
            cre_start = df_chains.pdb_start_cre.values[0]
            cre_end = df_chains.pdb_end_cre.values[0]
            if not kd_start < kd_end or not cre_start < cre_end:
              print(
                f"PDB: {pdb_id} has wrong kd or cre definitions. "
                f"Kd: [{kd_start, kd_end}, Cre: {cre_start, cre_end}]"
              )
              continue
            kd_range = set(range(kd_start, kd_end+1))
            cre_range = set(range(cre_start, cre_end+1))
            kd = pdb_positions.intersection(kd_range) ### toamr del mapeo
            cre = pdb_positions.intersection(cre_range)
            chains_kd_cre[chain] = {"kd": sorted(list(kd)), "cre": sorted(list(cre))}
          # check if output already exists
          os.makedirs(os.path.dirname(output_file), exist_ok = True)
          if not chains_kd_cre:
            continue
          if not os.path.exists(output_file):
            # print("inside path_output")
            # calculate distance
            # param pdb_source: a path to a pdb file, a Bio.PDB.Structure or a Bio.PDB.Model
            dist = distances.calculate_distances(
              pdb_source= f, # cambiar por archivo q no contenga los hetatoms
              chains_kd_cre= chains_kd_cre,
              atom_selector= all_atoms_selector,
              include_extra_info= True
            )
            print(f"distances for {pdb_id} calculated")
            # df with distances filtered by threshold
            df = pd.DataFrame(
              dist,
              columns = [
                'chain_a', 'pos_a', 'aa_a', 'atom_a',
                'chain_b', 'pos_b', 'aa_b', 'atom_b',
                'dist'
              ]
            )
            df = df[(df.pos_a > 0) & (df.pos_b > 0)]
            df["pdb"] = pdb_id
            print(f"writing distances as {pdb_id}.csv")
            df.to_csv(output_file, index= False)
      except BaseException as e:
        print(str(e))

In [11]:
# Generate a dataframe with the positions of CRE and KD in PDB
mapping_df = mapping_coordinates_uniprot_pdb[['uniprot', 'pdb', 'chain', 'mapping']]
regions_df = msa_regions_pdbs[['msa', 'uniprot', 'pdb','term_id_cre', 'start_cre', 'end_cre', 'term_id_kd', 'start_kd', 'end_kd']]

# Merge both tables based on uniprot-pdb
merged_df = regions_df.merge(mapping_df, on=["uniprot", "pdb"])
# Create columns for coordinates in pdb
merged_df["pdb_start_cre"] = 0
merged_df["pdb_end_cre"] = 0
merged_df["pdb_start_kd"] = 0
merged_df["pdb_end_kd"] = 0

pdbs_with_domains_not_ib_pdb = set()
for index, row in merged_df.iterrows():
    # if row["pdb"] != "5upk":
    #     continue
    start_cre = row["start_cre"]
    end_cre = row["end_cre"]
    start_kd = row["start_kd"]
    end_kd = row["end_kd"]
    # eval() function is used to convert the string representation into a dictionary object, which is stored in the mapping variable.
    mapping = eval(row["mapping"])
    min_mapping = min(mapping.keys())
    max_mapping = max(mapping.keys())

    # get the coordinates in pdb using the mapping
    # start look forward if NaN a new start, ends look backwards for a new end if NaN
    while start_cre <= max_mapping:
        pdb_start_cre = mapping.get(start_cre, np.nan)
        if np.isnan(pdb_start_cre) and start_cre < end_cre:
            start_cre += 1
        else:
            break

    while end_cre >= min_mapping:
        pdb_end_cre = mapping.get(end_cre, np.nan)
        if np.isnan(pdb_end_cre) and start_cre < end_cre:
            end_cre -= 1
        else:
            break

    if start_cre == end_cre:
        print(f"PDB: {row['pdb']}:{row['chain']} CRE not in pdb. Should be ignored.")
        pdbs_with_domains_not_ib_pdb.add(row["pdb"])

    while start_kd <= max_mapping:
        pdb_start_kd = mapping.get(start_kd, np.nan)
        if np.isnan(pdb_start_kd) and start_kd < end_kd:
            start_kd += 1
        else:
            break

    while end_kd >= min_mapping:
        pdb_end_kd = mapping.get(end_kd, np.nan)
        if np.isnan(pdb_end_kd) and start_kd < end_kd:
            end_kd -= 1
        else:
            break

    if start_kd == end_kd:
        print(f"PDB: {row['pdb']}:{row['chain']} KD domain not in pdb. Should be ignored.")
        pdbs_with_domains_not_ib_pdb.add(row["pdb"])

    # # Add a check to make sure that pdb_start_* is less than or equal to pdb_end_*
    # if pdb_start_cre > pdb_end_cre:
    #     pdb_start_cre, pdb_end_cre = pdb_end_cre, pdb_start_cre
    # if pdb_start_kd > pdb_end_kd:
    #     pdb_start_kd, pdb_end_kd = pdb_end_kd, pdb_start_kd

    # the coordinates in pdb are written back to their respective columns
    merged_df.at[index, "pdb_start_cre"] = pdb_start_cre
    merged_df.at[index, "pdb_end_cre"] = pdb_end_cre
    merged_df.at[index, "pdb_start_kd"] = pdb_start_kd
    merged_df.at[index, "pdb_end_kd"] = pdb_end_kd

PDB: 5upk:A KD domain not in pdb. Should be ignored.
PDB: 5upk:B CRE not in pdb. Should be ignored.
PDB: 4l67:A CRE not in pdb. Should be ignored.
PDB: 4fii:A CRE not in pdb. Should be ignored.
PDB: 4fii:B KD domain not in pdb. Should be ignored.
PDB: 4fif:A CRE not in pdb. Should be ignored.
PDB: 4fif:C KD domain not in pdb. Should be ignored.
PDB: 2hz0:A CRE not in pdb. Should be ignored.
PDB: 6npe:B CRE not in pdb. Should be ignored.
PDB: 6npu:B CRE not in pdb. Should be ignored.
PDB: 7cc2:A CRE not in pdb. Should be ignored.
PDB: 3cs9:A CRE not in pdb. Should be ignored.
PDB: 2g2h:A CRE not in pdb. Should be ignored.
PDB: 6bl8:A CRE not in pdb. Should be ignored.
PDB: 2v7a:A CRE not in pdb. Should be ignored.
PDB: 4twp:A CRE not in pdb. Should be ignored.
PDB: 4zog:A CRE not in pdb. Should be ignored.
PDB: 3ue4:A CRE not in pdb. Should be ignored.
PDB: 4yc8:A CRE not in pdb. Should be ignored.
PDB: 2hz4:A CRE not in pdb. Should be ignored.
PDB: 2hzi:A CRE not in pdb. Should be igno

In [8]:
print(list(pdbs_with_domains_not_ib_pdb))

['2v7a', '4qfs', '4twp', '7jhg', '4cfh', '1jks', '7m74', '4qfr', '3vut', '5ufu', '3qrj', '4qfg', '7n9g', '7jhh', '5kq5', '2hz0', '3cs9', '1p4f', '5upk', '2hz4', '4fii', '2g2h', '6npe', '6bl8', '4fif', '1jkl', '3ue4', '5t5t', '2hzi', '7dt2', '6e4t', '6npu', '4zog', '4yc8', '1f3m', '2j0m', '4l67', '6e4w', '1ig1', '6c9f', '6e4u', '2hyy', '7cc2']


---

### Cálculo de distancias

In [21]:
to_calculate =  merged_df.dropna().copy()

In [22]:
print(
  f"There are {to_calculate.pdb.nunique()} PDB structures, "
  f"corresponding to {to_calculate.uniprot.nunique()} proteins "
  f"in {to_calculate.msa.nunique()} MSAs"
)

There are 801 PDB structures, corresponding to 48 proteins in 56 MSAs


In [23]:
to_calculate.pdb_start_cre = to_calculate.pdb_start_cre.apply(int)
to_calculate.pdb_end_cre = to_calculate.pdb_end_cre.apply(int)
to_calculate.pdb_start_kd = to_calculate.pdb_start_kd.apply(int)
to_calculate.pdb_end_kd = to_calculate.pdb_end_kd.apply(int)

In [24]:
wrong_calculated_pdbs = [
  "3d42", "2oza", "4red", "4o2p", "3w2p", "6bab", "4r3r", "3gt8", "3gop",
  "3e88", "6e4t", "2jdr", "2hwp", "7jxp", "4riw", "7opm", "6nyb", "7m74",
  "3r2b", "4rew", "4fif", "6c9f", "7m0x", "5hib", "7mfd", "3eqd", "4rer",
  "3u51", "3cqw", "2uw9", "4tyh", "4qfs", "5edp", "6npz", "5upk", "4ekk",
  "1jkl", "5uwd", "6z4d", "4zhx", "4ump", "7m0v", "5jeb", "6paw", "2jam",
  "3tei", "3eqb", "3ocb", "3wzd", "2jdo", "5v62", "6e4u", "2rfe", "2gs6",
  "2x0g", "7tvd", "4riy", "4cfh", "7m0w", "4rix", "2jbp", "7oxb", "5bvd",
  "7jhh", "5bmm", "5ufu", "3cqu", "3e87", "7jij", "3alo", "6tca", "6c9h",
  "4asd", "6ao5", "3d7u", "4fii", "4ixp", "3ow4", "4l67", "4iza", "1jks",
  "3g5d", "5czi", "2g1t", "7m0u", "5v61", "4h3p", "6c9g", "3qkl", "4anb",
  "1y6b", "1kwp", "2hz4", "5fed", "2g2f", "4cfe", "4g5p", "6q0j", "5nhp",
  "4fmq", "3qkk", "6mob", "3lcd", "5iso", "6z4b", "6v5p", "4fgb", "6gjb",
  "4mne", "5kq5", "4h3q", "6p1d", "7jhg", "1rjb", "3mv5", "6g9d", "6b1u",
  "4iz5", "6v2w", "6b2e", "2wel", "2rgp", "5czh", "4qfr", "3u4w", "2rf9",
  "4agd", "4qfg", "3e8d", "5t0p", "2x39", "3mvh", "7m0y", "6u2g", "5t5t",
  "6q0t", "3lzb", "6buu", "5gmp", "6xvb", "7nrb", "4r3p", "6c9j", "4zjv",
  "1ig1", "7myj", "6e4w", "2xh5", "5ugb", "5ezv", "3cpb", "2e2b", "1p4f",
  "2itp", "4lgd", "4cff", "3vid", "6vhp", "4twp", "7m0t", "3d44", "4d2p",
  "3d7t", "2g2i", "6s9d", "7apj", "4i21", "7m0z", "5ax3", "6fhb", "6gqp",
  "2onl", "2yab", "1o6l", "2rfd", "6lud", "6lub", "2y9q", "5cap", "6slg",
  "1o6k", "6gjd", "6pp9", "6s9b"
]

In [26]:
good_case = (
  to_calculate
    .loc[~to_calculate.pdb.isin(wrong_calculated_pdbs), :]
    .iloc[[0], :]
)


In [27]:
domains_outside_pdbs = [
  '7dt2', '2v7a', '4twp', '1jks', '5ufu', '2j0m', '6c9f', '2hz0', '6e4t',
  '4fif', '2hzi', '3ue4', '7jhg', '4zog', '7n9g', '6e4w', '4yc8', '7cc2',
  '4qfs', '2hz4', '1ig1', '5t5t', '4cfh', '5kq5', '2g2h', '2hyy', '5upk',
  '7m74', '4l67', '1jkl', '4qfg', '6e4u', '7jhh', '6npu', '4fii', '1f3m',
  '4qfr', '1p4f', '3qrj', '6bl8', '3cs9', '6npe', '3vut'
]

In [28]:
wrong_calculated_pdbs = list(
  set(wrong_calculated_pdbs).difference(domains_outside_pdbs)
)

In [29]:
bad_case = (
  to_calculate
    .loc[to_calculate.pdb.isin(wrong_calculated_pdbs), :]
    .iloc[[0], :]
)

bad_cases = (
  to_calculate
    .loc[to_calculate.pdb.isin(wrong_calculated_pdbs), :]
)

In [30]:
distances_all_atoms(
  path= '../datasets/pdb_files/',
  output= '../datasets/interatomic_distances_both_regions_kd_cre/',
  pdb_coord = bad_cases
)

distances for 3eqb calculated
writing distances as 3eqb.csv
distances for 4mne calculated
writing distances as 4mne.csv
distances for 4lgd calculated
writing distances as 4lgd.csv
distances for 6ao5 calculated
writing distances as 6ao5.csv
distances for 2wel calculated
writing distances as 2wel.csv
distances for 6bab calculated
writing distances as 6bab.csv
distances for 4fgb calculated
writing distances as 4fgb.csv
distances for 4d2p calculated
writing distances as 4d2p.csv
distances for 4ump calculated
writing distances as 4ump.csv
distances for 4ixp calculated
writing distances as 4ixp.csv
distances for 7nrb calculated
writing distances as 7nrb.csv
distances for 2yab calculated
writing distances as 2yab.csv
distances for 6paw calculated
writing distances as 6paw.csv
distances for 2jam calculated
writing distances as 2jam.csv


In [14]:
msa_regions_pdbs.msa.value_counts()

P00533_60    215
P28482_60    123
P00523_60     70
Q02750_60     60
P35968_60     42
P00519_60     39
Q14680_60     31
P31749_60     28
P49137_60     26
P49138_60     26
Q13131_60     22
P54645_60     22
P10721_60     22
P07333_60     19
P31751_60     16
Q80YE7_60     15
P53355_60     15
Q8VDF3_60      9
Q09137_60      8
Q8BRK8_60      8
P54646_60      8
O75582_60      7
A5K0N4_60      7
Q8I719_60      7
P36888_60      6
Q9UIK4_60      6
O96013_60      5
P43403_60      5
Q8IU85_60      5
Q00944_60      5
Q8BW96_60      5
Q91YS8_60      5
Q13188_60      5
Q16644_60      5
P51812_60      5
Q13557_60      5
Q14012_60      4
P45985_60      3
Q13043_60      3
Q15418_60      3
Q01974_60      2
Q8IBS5_60      2
Q61846_60      2
Q6PHZ2_60      1
Q16566_60      1
Q91VB2_60      1
Q08E52_60      1
Q13153_60      1
Q13976_60      1
Q13555_60      1
P29323_60      1
O74536_60      1
P04049_60      1
Q96NX5_60      1
Q13554_60      1
Q63450_60      1
P36507_60      1
O77676_60      1
Name: msa, dty